In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from datetime import datetime
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import json

from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

In [2]:
datetime.now().strftime("%m-%d-%Y_%Hh%Mmin%Ss")

'07-17-2020_14h16min49s'

In [3]:
global_epoch_number = 30

## Reading data

In [4]:
# reading data
evo_data = pd.read_csv('data/interpol/evo_interpol_demand.csv', index_col=0)
modo_data = pd.read_csv('data/interpol/modo_interpol_demand.csv', index_col=0)
c2g_data = pd.read_csv('data/interpol/c2g_interpol_demand.csv', index_col=0)

In [5]:
evo_data.columns

Index(['tempC', 'precipMM', 'FeelsLikeC', 'uvIndex', 'visibility',
       'windspeedMiles', 'Blizzard', 'Clear', 'Cloudy', 'Fog', 'Heavy rain',
       'Heavy rain at times', 'Heavy snow', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light sleet showers', 'Light snow',
       'Mist', 'Moderate or heavy freezing rain',
       'Moderate or heavy rain shower', 'Moderate or heavy rain with thunder',
       'Moderate or heavy sleet', 'Moderate or heavy snow showers',
       'Moderate or heavy snow with thunder', 'Moderate rain',
       'Moderate rain at times', 'Moderate snow', 'Overcast', 'Partly cloudy',
       'Patchy heavy snow', 'Patchy light drizzle', 'Patchy light rain',
       'Patchy light rain with thunder', 'Patchy light snow',
       'Patchy moderate snow', 'Patchy rain possible', 'Patchy sleet possible',
       'Patchy snow possible', 'Sunny', 'Thundery outbreaks possible',
       'Torrential rain shower', 'Monday', 'Tuesday', 'Wednesday', 'Thursday

In [6]:
evo_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'], inplace=True)
modo_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'], inplace=True)
c2g_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'], inplace=True)

In [7]:
unievo_data = pd.DataFrame(evo_data.travels)
unimodo_data = pd.DataFrame(modo_data.travels)
unic2g_data = pd.DataFrame(c2g_data.travels)

In [8]:
# Adding the Canada Day Holiday
evo_data.index = pd.to_datetime(evo_data.index)
modo_data.index = pd.to_datetime(modo_data.index)
c2g_data.index = pd.to_datetime(c2g_data.index)

evo_data["holidays"] = pd.Series()
modo_data["holidays"] = pd.Series()
c2g_data["holidays"] = pd.Series()

evo_data["holidays"] = evo_data["holidays"].fillna(0)
modo_data["holidays"] = modo_data["holidays"].fillna(0)
c2g_data["holidays"] = c2g_data["holidays"].fillna(0)

canada_day = datetime(2018, 7, 1)
end_canada_day = datetime(2018,7 ,3)

evo_data.loc[((evo_data.index > canada_day) & (evo_data.index <= end_canada_day))]["holidays"] = 1
modo_data.loc[((modo_data.index > canada_day) & (modo_data.index <= end_canada_day))]["holidays"] = 1
c2g_data.loc[((c2g_data.index > canada_day) & (c2g_data.index <= end_canada_day))]["holidays"] = 1

In [9]:
init_period = '06-23'
end_period = '07-12'

evo_data = evo_data[(evo_data.index >= '2018-'+init_period) & (evo_data.index <= '2018-'+end_period)]
modo_data = modo_data[(modo_data.index >= '2018-'+init_period) & (modo_data.index <= '2018-'+end_period)]
c2g_data = c2g_data.loc["2016-12-13 15:00:00":"2017-02-25 17:00:00"]

unievo_data = unievo_data[(unievo_data.index >= '2018-'+init_period) & (unievo_data.index <= '2018-'+end_period)]
unimodo_data = unimodo_data[(unimodo_data.index >= '2018-'+init_period) & (unimodo_data.index <= '2018-'+end_period)]
unic2g_data = unic2g_data.loc["2016-12-13 15:00:00":"2017-02-25 17:00:00"]

## LSTM Data Preparation

In [10]:
def sup_learning_formatter(data, past_lags, future_steps, train_split):
    X = []
    y = []

    norm_data  = data.values

    for n in range(len(data) - past_lags - future_steps):
        X.append(norm_data[n : n + past_lags])
        y.append(data.travels.values[n + past_lags : n + past_lags + future_steps])
    return np.array(X), np.array(y)

In [11]:
def train_val_test_splitter(data, splits):
    locs = [int(len(data)*n) for n in splits]
    return data[:locs[0]], data[locs[0]:locs[1]], data[locs[1]:], data[0].shape

In [12]:
def eval_model(y, y_hat):
    evaluation = {}
    evaluation["RMSE"] = np.sqrt(mean_squared_error(y, y_hat))
    evaluation["MAE"] = mean_absolute_error(y, y_hat)
    evaluation["R2"] = r2_score(y, y_hat)

    return evaluation

In [13]:
def persistance_model(X, timesteps):
    y_hat = []
    for x in X:
        y_hat.append(np.array([x[-1][0] for _ in range(timesteps)]))

    return np.array(y_hat)

# Training Models

In [14]:
def plot_train_history(history, title, save_file=False):
    history = pd.DataFrame(history.history)

    history.plot(figsize=(8, 5))
    plt.grid(True)
    plt.savefig("plots\\" + title.replace(" ", "_") + ".png", bbox_inches='tight') if save_file else print()
    plt.show()

Next, will be generated the model for each dataset

## Grid Search

In [15]:
class GridSearchLSTM:
    def __init__(self):
        self.evaluations = pd.DataFrame()
        self.best_estimator = None

    def search(self, feature_dict, data, verbose=1, windows=1, splits = (0.6, 0.8)):
        
        def average_evaluations(validation_eval, key="val_loss"):
            acc_value = 0
            for split, evaluation in validation_eval:
                acc_value += evaluation[key]
            return acc_value/len(validation_eval)
        
        possibilities_list = self._create_feature_dict(feature_dict)
        current_evaluations = []
        if(windows == 1):
            for test in tqdm(possibilities_list):
                model, hist, test_data, evaluation = self.run_lstm(data, 24, 12, splits, verbose=verbose, **test)
                validation_eval = {key:value[-1] for key, value in hist.history.items()}
                current_evaluations.append([test, validation_eval])
        
        else:
            increase = splits[1]/(windows + 1)
            for test in tqdm(possibilities_list):
                validation_eval = []
                for i in tqdm(range(windows)):
                    cur_split = (increase*(i + 1), increase*(i + 2))
                    model, hist, test_data, evaluation = self.run_lstm(data, 24, 12, cur_split, verbose=verbose, **test)
                    cur_validation_eval = {key:value[-1] for key, value in hist.history.items()}
                    validation_eval.append([cur_split, cur_validation_eval])
                    print(cur_split)
                current_evaluations.append([test, validation_eval])
        
        if(windows == 1):
            current_evaluations.sort(key=lambda x: x[1]["val_loss"])
        else:
            current_evaluations.sort(key=lambda x: average_evaluations(x[1]))   
        
        self.evaluations = pd.DataFrame(map(lambda x: {**x[0], **x[1]}, current_evaluations))
        self.best_estimator = current_evaluations[0][0]
            

    def _create_feature_dict(self, feature_dict):
        return self._create_feature_dict_recurse({}, feature_dict, list(feature_dict.keys()))

    def _create_feature_dict_recurse(self, start_dict, feature_dict, remaining_keys):
        if len(remaining_keys) == 0:
            return [start_dict]
        new_feature_dict = feature_dict.copy()
        returned_list = []
        del new_feature_dict[remaining_keys[0]]
        for item in feature_dict[remaining_keys[0]]:
            new_start_dict = start_dict.copy()
            new_start_dict[remaining_keys[0]] = item
            returned_list += self._create_feature_dict_recurse(new_start_dict, new_feature_dict, remaining_keys[1:])
        return returned_list


    def run_lstm(self, data, past_lags, future_steps, splits, node_number=50,
                 epochs=10, batch_size=64, loss='mae', dropout=0.5, layer_count=2, verbose=1):
        
        X, y = sup_learning_formatter(data, past_lags, future_steps, splits[0])
        X_train, X_val, X_test, X_shape = train_val_test_splitter(X, splits)
        y_train, y_val, y_test, y_shape = train_val_test_splitter(y, splits)


        train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        train = train.cache().shuffle(batch_size).batch(batch_size).repeat()

        val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        val = val.batch(batch_size).repeat()

        model = tf.keras.models.Sequential()

        if(layer_count == 1):
            model.add(tf.keras.layers.LSTM(node_number,
                                    input_shape=X_shape))
            model.add(tf.keras.layers.Dropout(dropout))
        else:
            model.add(tf.keras.layers.LSTM(node_number, return_sequences=True,
                                    input_shape=X_shape))
            model.add(tf.keras.layers.Dropout(dropout))

            for _ in range(layer_count - 2):
                model.add(tf.keras.layers.LSTM(node_number, return_sequences=True, activation='relu'))

            model.add(tf.keras.layers.LSTM(node_number, activation='relu'))
        
        model.add(tf.keras.layers.Dense(12))
        
        def rmse(y_true, y_pred):
            return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))

        model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss=loss, metrics=[rmse])
        
        history = model.fit(train, epochs=epochs, steps_per_epoch=50,
                            validation_data=val, validation_steps=50, verbose=verbose
                            )
        y_hat_test = model.predict(X_test)
        evaluation = eval_model(y_test, y_hat_test)

        return model, history, (X_test, y_test), evaluation


In [16]:
grid_search = GridSearchLSTM()

## Test Windows

In [39]:
feature_dict = {"epochs":[25], "layer_count":[2, 3, 4, 5], "node_number":[160, 180, 200, 220], "dropout":[0.3, 0.7]}

grid_search.search(feature_dict, unic2g_data, windows=1)

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 6s 126ms/step - loss: 126.5316 - rmse: 158.0425 - val_loss: 68.7451 - val_rmse: 84.4685
Epoch 2/25
50/50 [==============================] - 5s 107ms/step - loss: 73.1685 - rmse: 90.5638 - val_loss: 72.9477 - val_rmse: 90.8810
Epoch 3/25
50/50 [==============================] - 4s 76ms/step - loss: 57.6472 - rmse: 72.7321 - val_loss: 45.1816 - val_rmse: 56.6637
Epoch 4/25
50/50 [==============================] - 5s 93ms/step - loss: 49.5288 - rmse: 63.8220 - val_loss: 40.0666 - val_rmse: 52.0412
Epoch 5/25
50/50 [==============================] - 4s 87ms/step - loss: 44.5447 - rmse: 58.1149 - val_loss: 40.4173 - val_rmse: 53.2434
Epoch 6/25
50/50 [==============================] - 7s 133ms/step - loss: 40.8660 - rmse: 53.7451 - val_loss: 44.9528 - val_rmse: 57.5953
Epoch 7/25
50/50 [==============================] - 8s 165ms/step - loss: 39.9096 - rmse: 52.4519 - val_loss: 37.9269 - val_rmse: 5

In [40]:
def evaluator(x):
    return sum([splits[-1]*values['val_loss'] for (splits, values) in x])

In [41]:
grid_search.evaluations

,epochs,layer_count,node_number,dropout,loss,rmse,val_loss,val_rmse
0,25,3,160,0.3,24.845056,33.734009,26.676975,36.386173
1,25,2,160,0.3,25.494295,34.789776,26.759470,36.620213
2,25,3,180,0.7,26.824999,36.214920,26.982515,36.409119
3,25,2,180,0.8,28.288883,38.149914,27.069027,36.188385
4,25,2,180,0.7,27.182920,36.467155,27.242572,36.849945
5,25,3,160,0.7,26.703194,36.160015,27.409861,37.074490
6,25,2,160,0.7,26.116151,35.460846,27.493419,36.974094
7,25,2,180,0.5,25.228306,34.108627,27.522308,37.175056
8,25,2,100,0.3,26.567664,35.971237,27.772507,37.656372
9,25,2,160,0.8,28.034410,37.920330,28.028752,37.688915


In [42]:
grid_search.evaluations.to_csv(f'results\\GridSearch_Results\\unic2g_grid_search_{datetime.now().strftime("%m-%d-%Y_%Hh%Mmin%Ss")}.csv')

In [17]:
feature_dict = {"epochs":[25], "layer_count":[2, 3], "node_number":[120, 140, 160, 180, 200, 220, 240], "dropout":[0.3, 0.7]}

grid_search.search(feature_dict, c2g_data, windows=1)

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 9s 177ms/step - loss: 332.6501 - rmse: 424.0037 - val_loss: 81.8265 - val_rmse: 100.3055
Epoch 2/25
50/50 [==============================] - 5s 92ms/step - loss: 72.2040 - rmse: 89.4203 - val_loss: 90.0662 - val_rmse: 110.4281
Epoch 3/25
50/50 [==============================] - 4s 90ms/step - loss: 65.8759 - rmse: 82.4931 - val_loss: 71.3898 - val_rmse: 89.5847
Epoch 4/25
50/50 [==============================] - 4s 85ms/step - loss: 55.8453 - rmse: 71.8360 - val_loss: 51.2950 - val_rmse: 68.0891
Epoch 5/25
50/50 [==============================] - 4s 87ms/step - loss: 48.0122 - rmse: 62.3496 - val_loss: 59.1287 - val_rmse: 78.4267
Epoch 6/25
50/50 [==============================] - 5s 106ms/step - loss: 43.6423 - rmse: 57.1531 - val_loss: 45.0024 - val_rmse: 59.2179
Epoch 7/25
50/50 [==============================] - 4s 78ms/step - loss: 41.3557 - rmse: 54.1416 - val_loss: 47.5573 - val_rmse: 6

In [18]:
grid_search.evaluations

,epochs,layer_count,node_number,dropout,loss,rmse,val_loss,val_rmse
0,25,2,220,0.3,18.508599,24.894522,22.956626,31.818382
1,25,3,240,0.3,18.981148,25.556908,24.707572,34.186726
2,25,2,240,0.7,20.642672,27.936657,24.743443,34.534412
3,25,2,160,0.3,20.177676,27.377291,24.779557,33.905300
4,25,2,180,0.7,21.777988,29.130457,24.861382,34.680988
5,25,3,140,0.3,22.008420,29.407026,25.019000,33.984753
6,25,3,220,0.7,19.742950,26.973797,25.056960,33.999569
7,25,3,180,0.7,21.165813,28.698973,25.384185,34.750221
8,25,3,140,0.7,21.585495,28.925949,25.476516,34.769283
9,25,3,220,0.3,19.414039,26.028904,25.520435,34.795708


In [19]:
grid_search.evaluations.to_csv(f'results\\GridSearch_Results\\c2g_grid_search_{datetime.now().strftime("%m-%d-%Y_%Hh%Mmin%Ss")}.csv')